In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import scale
from sklearn.preprocessing import normalize
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

# Preparing Data

In [2]:
SAMPLE_SIZE = 50
NUM_PREDICTORS = 20
NOISE_VAR = 16

X = np.random.rand(SAMPLE_SIZE, NUM_PREDICTORS)
X_scaled = scale(X, axis=0)
X_norm = normalize(X, norm='l2', axis=0)

In [3]:
y = (10 * X[:, 0] + 5 * X[:, 1] + 0.1 * X[:, 2] + 
     np.random.normal(loc=0, scale=NOISE_VAR, size=1)).reshape(SAMPLE_SIZE,1)
y_scaled = scale(y, axis=0)
y_norm = normalize(y, norm='l2', axis=0)

# Least Squares Linear Regression

In [4]:
clf = LinearRegression(normalize=True)
clf.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)

In [5]:
clf.coef_

array([[  1.00000000e+01,   5.00000000e+00,   1.00000000e-01,
         -5.79977654e-15,  -3.45388712e-15,  -2.03718869e-15,
          5.91394417e-15,  -6.43438348e-15,   1.23031185e-14,
          2.29947584e-15,  -4.21159748e-15,  -9.08037536e-15,
          1.47448827e-17,   5.66219376e-15,  -1.91097935e-15,
         -9.36584937e-15,   2.17089241e-15,   3.67548749e-15,
         -8.45752243e-16,  -3.16318466e-15]])

# Lasso

In [6]:
def lasso(alpha, X, y, normalize=True):
    clf_lasso = Lasso(alpha=alpha, normalize=normalize)
    clf_lasso.fit(X, y)
    return clf_lasso.coef_

In [7]:
alphas = [0.001, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]

d = []
for alpha in alphas:
    coeffs = lasso(alpha, X, y, normalize=False)
    d.append({'alpha': alpha, 
              'beta1': coeffs[0], 
              'beta2': coeffs[1],
              'beta3': coeffs[2]})

df_lasso = pd.DataFrame(d, columns=['alpha', 'beta1', 'beta2', 'beta3'])

In [8]:
df_lasso

,alpha,beta1,beta2,beta3
0,0.001,9.985616,4.987368,0.091867
1,0.010,9.855772,4.873443,0.018726
2,0.050,9.266324,4.295353,0.000000
3,0.100,8.528566,3.567257,0.000000
4,0.150,7.790808,2.839160,0.000000
5,0.200,7.053050,2.111064,0.000000
6,0.250,6.315291,1.382968,0.000000
7,0.300,5.577531,0.654871,0.000000


# Effect of Scaling on Coefficients

In [9]:
# First divide the first column of X by 10
X_prime = X.copy()
X_prime[:, 0] *= 1000.0

coef = lasso(alpha=0.001, X=X_prime, y=y, normalize=False)
print(coef)

[ 0.00999924  4.98857766  0.09141308 -0.          0.          0.         -0.
 -0.         -0.          0.          0.         -0.          0.         -0.
  0.          0.         -0.          0.          0.          0.        ]


In [10]:
# First divide the first column of X by 10
X_prime = X.copy()
X_prime[:, 0] /= 1000.0

coef = lasso(alpha=0.001, X=X_prime, y=y, normalize=True)
print(coef)

[  9.97238866e+03   4.97612242e+00   8.17459949e-02  -0.00000000e+00
   0.00000000e+00  -0.00000000e+00   0.00000000e+00  -0.00000000e+00
  -0.00000000e+00   0.00000000e+00   0.00000000e+00  -0.00000000e+00
   0.00000000e+00  -0.00000000e+00   0.00000000e+00   0.00000000e+00
  -0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]


# Ridge

In [11]:
def ridge(alpha, X, y):
    clf_ridge = Ridge(alpha, normalize=True)
    clf_ridge.fit(X, y)
    return clf_ridge.coef_    

In [12]:
alphas = [0.001, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]

d = []
for alpha in alphas:
    coeffs = ridge(alpha, X, y)
    d.append({'alpha': alpha, 
              'beta1': coeffs[0][0], 
              'beta2': coeffs[0][1],
              'beta3': coeffs[0][2]})

df_ridge = pd.DataFrame(d, columns=['alpha', 'beta1', 'beta2', 'beta3'])

In [13]:
df_ridge

,alpha,beta1,beta2,beta3
0,0.001,9.985133,4.992402,0.099306
1,0.010,9.854219,4.925035,0.093991
2,0.050,9.326979,4.646817,0.084603
3,0.100,8.765733,4.342618,0.089472
4,0.150,8.284841,4.078715,0.100918
5,0.200,7.865175,3.847691,0.113990
6,0.250,7.493941,3.643672,0.126779
7,0.300,7.162053,3.462062,0.138567
